**Exercise 1:Spell Check**

**Identify the list of candidate words using Trie structure for the misspelled word. Find the minimum
edit distance and choose the correct word based on the context.
INPUT:
"During the summer we have the best ueather.", "I have a black ueather jacket, so nice."
OUTPUT:
"During the summer we have the best weather.", "I have a black leather jacket, so nice."
Use the developed model to choose the correct spellings for the following sentences:**
1. Mr Patrick is our new (principle/principal).
2. The company (excepted/accepted) all the terms.
3. Please don’t keep your dog on the (lose/loose).
4. The (later/latter) is my best friend.
5. I need some (stationary/stationery) products for my craftwork.
6. The actor (excepted/accepted) the Oscar.
7. I will call you (later/latter) in the evening.
8. Covid (affects/effects) the lungs.
9. The (council/counsel) of the ministers were sworn in yesterday.
10. Robert (too/to) wants to accompany us to the park.
11. Mia will (council/counsel) me about choosing fashion as my career.
12. The (bear/bare) at the zoo was very playful.
13. The sheep have a lot of (fur/far) that keeps them warm.
14. The hot spring is at the (furthest/ farthest) corner of the street.
15. Can you (advice/advise) me on how to study for exams?
16. The team will (loose/lose) the match if they don’t play well.
17. Can you go (to/too) the market for me?
18. The teachers asked the students to keep (quite/quiet).
19. The (heap/hip) of garbage should be cleaned immediately.
20. This is (there/their) house.

**Exercise 2: Sentiment Analysis Dataset**



Task:
*  Explain the pipeline for developing sentiment analysis task.
*  Perform cleaning and preprocessing of text.
*  Generate representations using:
*  Bag of Words
*  TF-IDF
*  Continuous Bag of Words
*  Skip gram
*  Word2Vec.

1.  Classify the data using appropriate machine learning techniques to generate labels.
2.  Analyze the labels and explain the impact of embedding techniques in misclassification.
3.  Discuss the limitations of each embedding technique and explain the techniques that rectify it.**

Input:
What is not to like about this product.
Not bad.
Not an issue.
Not buggy.
Not happy.
Not user-friendly.
Not good.
Is it any good?
I do not dislike horror movies.
Disliking horror movies is not uncommon.
Sometimes I really hate the show.
I love having to wait two months for the next series to come out!
The final episode was surprising with a terrible twist at the end.
The film was easy to watch but I would not recommend it to my friends.
I LOL’d at the end of the cake scene

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('/content/train.csv', encoding='latin1')
test_df = pd.read_csv('/content/test.csv', encoding='latin1')

In [ ]:
train_df.shape
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [ ]:
#Remove unneccessary columns

columns_to_remove = ['textID','Time of Tweet', 'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)']
train_df = train_df.drop(columns=columns_to_remove)

In [ ]:
train_df = train_df.dropna()

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27480 entries, 0 to 27480
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           27480 non-null  object
 1   selected_text  27480 non-null  object
 2   sentiment      27480 non-null  object
dtypes: object(3)
memory usage: 858.8+ KB


In [ ]:
test_df.shape
test_df.info()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            3534 non-null   object 
 1   text              3534 non-null   object 
 2   sentiment         3534 non-null   object 
 3   Time of Tweet     3534 non-null   object 
 4   Age of User       3534 non-null   object 
 5   Country           3534 non-null   object 
 6   Population -2020  3534 non-null   float64
 7   Land Area (Km²)   3534 non-null   float64
 8   Density (P/Km²)   3534 non-null   float64
dtypes: float64(3), object(6)
memory usage: 338.7+ KB


,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [ ]:
columns_to_remove = ['textID','Time of Tweet', 'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)']
test_df = test_df.drop(columns=columns_to_remove)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3534 non-null   object
 1   sentiment  3534 non-null   object
dtypes: object(2)
memory usage: 75.4+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Handle missing values in the sentiment column
train_df = train_df.dropna(subset=["sentiment"])
test_df = test_df.dropna(subset=["sentiment"])

# Initialize LabelEncoder
encoder = LabelEncoder()

# Fit and transform labels in the training dataset
train_y = encoder.fit_transform(train_df["sentiment"])
test_y = encoder.transform(test_df["sentiment"])

Data Cleaning and Preprocessing

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def pre_processing(tweet: str):
    # Remove Leading Blank Spaces
    tweet = tweet.strip()

    # Lower Case
    tweet = tweet.lower()

    # Remove URLS
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    tweet = re.sub(url_pattern, "", tweet)

    # Remove UserName
    username_pattern = re.compile(r"@\w+")
    tweet = re.sub(username_pattern, "", tweet)

    # Remove Hashtags
    hashtag_pattern = re.compile(r"#\w+")
    tweet = re.sub(hashtag_pattern, "", tweet)

    # Character normalization // todaaaaay -> today
    tweet = re.sub(r"([a-zA-Z])\1{2,}", r'\1', tweet)

    # Remove Special Characters
    tweet = re.sub(r'[^a-zA-Z\s]', "", tweet)

    # Word Tokenizer
    tweet = nltk.word_tokenize(tweet)

    # Lemmatization and Part-of-Speech tagging
    def get_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"N": "n", "V": "v", "R": "r", "J": "a"}
        return tag_dict.get(tag, "n")

    lemma = WordNetLemmatizer()
    tweet = [lemma.lemmatize(word, pos=get_pos(word)) for word in tweet]

    return tweet

# Example usage of pre_processing function
processed_tweet = pre_processing("I loveeeee NLP, @apple_apple, www.apple_x.com, #NLP ")
print(processed_tweet)


['i', 'love', 'nlp']


In [ ]:
train_df["pre-tweet"] = train_df["text"].apply(pre_processing)
test_df["pre-tweet"] = test_df["text"].apply(pre_processing)

train_df["documents"] = train_df["pre-tweet"].apply(lambda x : " ".join(x))
test_df["documents"] = test_df["pre-tweet"].apply(lambda x : " ".join(x))



vocab = set()

for words in train_df["pre-tweet"]:
    for word in words:
        vocab.add(word)

print("Vocab Size :", len(vocab))

Vocab Size : 22037


Generate representations using:
• Bag of Words
• TF-IDF
• Continuous Bag of Words
• Skip gram
• Word2Vec.

Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bag_of_words = CountVectorizer()

train_bow = bag_of_words.fit_transform(train_df["documents"])
test_bow = bag_of_words.transform(test_df["documents"])

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_bow, train_y)


from sklearn.metrics import classification_report, accuracy_score

predict = model.predict(test_bow)
print("Accuracy Score :", accuracy_score(test_y, predict), end='\n\n')
print(classification_report(y_true = test_y, y_pred = predict))

Accuracy Score : 0.7023203169213356

              precision    recall  f1-score   support

           0       0.71      0.64      0.67      1001
           1       0.64      0.73      0.68      1430
           2       0.79      0.72      0.75      1103

    accuracy                           0.70      3534
   macro avg       0.71      0.70      0.70      3534
weighted avg       0.71      0.70      0.70      3534



Tf-Idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()

train_idf = tf_idf.fit_transform(train_df["documents"])
test_idf = tf_idf.transform(test_df["documents"])

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_idf, train_y)

from sklearn.metrics import classification_report, accuracy_score

predict = model.predict(test_idf)
print("Accuracy Score :", accuracy_score(test_y, predict), end='\n\n')
print(classification_report(y_true = test_y, y_pred = predict))

Accuracy Score : 0.7105263157894737

              precision    recall  f1-score   support

           0       0.73      0.65      0.69      1001
           1       0.64      0.75      0.69      1430
           2       0.81      0.71      0.76      1103

    accuracy                           0.71      3534
   macro avg       0.73      0.70      0.71      3534
weighted avg       0.72      0.71      0.71      3534



CBOW

In [ ]:
from gensim.models import Word2Vec

g_model = Word2Vec(sentences=train_df["pre-tweet"], vector_size=200, window=5, workers=5, epochs=500)
def in_vocab(word_l):
    for word in word_l:
        if word not in g_model.wv:
            return False
    else:
        return True

train_vec = [g_model.wv[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((200)) for x in train_df["pre-tweet"]]
test_vec  = [g_model.wv[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((200)) for x in test_df["pre-tweet"]]
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_vec, train_y)

from sklearn.metrics import classification_report, accuracy_score

predict = model.predict(test_vec)
print("Accuracy Score :", accuracy_score(test_y, predict), end='\n\n')
print(classification_report(y_true = test_y, y_pred = predict))

Accuracy Score : 0.5249009620826259

              precision    recall  f1-score   support

           0       0.66      0.29      0.40      1001
           1       0.47      0.86      0.60      1430
           2       0.74      0.30      0.43      1103

    accuracy                           0.52      3534
   macro avg       0.62      0.48      0.48      3534
weighted avg       0.61      0.52      0.49      3534



Skip gram

In [ ]:
from gensim.models import Word2Vec

g_model = Word2Vec(sentences=train_df["pre-tweet"], vector_size=200, window=5, workers=5, sg=1, epochs=500)
def in_vocab(word_l):
    for word in word_l:
        if word not in g_model.wv:
            return False
    else:
        return True

train_vec = [g_model.wv[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((200)) for x in train_df["pre-tweet"]]
test_vec  = [g_model.wv[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((200)) for x in test_df["pre-tweet"]]
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_vec, train_y)

from sklearn.metrics import classification_report, accuracy_score

predict = model.predict(test_vec)
print("Accuracy Score :", accuracy_score(test_y, predict), end='\n\n')
print(classification_report(y_true = test_y, y_pred = predict))

KeyboardInterrupt: 

Word2vec

In [ ]:
import gensim.downloader as api

model = api.load("glove-twitter-200")

shape_n = 200

def in_vocab(word_l):
    for word in word_l:
        if word not in model:
            return False
    else:
        return True

train_vec = [model[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((shape_n)) for x in train_df["pre-tweet"]]
test_vec  = [model[x].sum(axis = 0) if len(x) and in_vocab(x) else np.zeros((shape_n)) for x in test_df["pre-tweet"]]
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_vec, train_y)

from sklearn.metrics import classification_report, accuracy_score

predict = model.predict(test_vec)
print("Accuracy Score :", accuracy_score(test_y, predict), end='\n\n')
print(classification_report(y_true = test_y, y_pred = predict))

[==================================================] 100.0% 758.5/758.5MB downloaded
Accuracy Score : 0.642331635540464

              precision    recall  f1-score   support

           0       0.72      0.56      0.63      1001
           1       0.57      0.73      0.64      1430
           2       0.73      0.60      0.66      1103

    accuracy                           0.64      3534
   macro avg       0.67      0.63      0.64      3534
weighted avg       0.66      0.64      0.64      3534



In [ ]:

text = """What is not to like about this product.
Not bad.
Not an issue.
Not buggy.
Not happy.
Not user-friendly.
Not good.
Is it any good?
I do not dislike horror movies.
Disliking horror movies is not uncommon.
Sometimes I really hate the show.
I love having to wait two months for the next series to come out!
The final episode was surprising with a terrible twist at the end.
The film was easy to watch but I would not recommend it to my friends.
I LOL’d at the end of the cake scene."""

input_text = text.split("\n")
input_text = [" ".join(pre_processing(string)) for string in input_text]
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()

train_idf = tf_idf.fit_transform(train_df["documents"])
pred_idf = tf_idf.transform(input_text)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(train_idf, train_y)

predict = model.predict(pred_idf)
predict = encoder.inverse_transform(predict)
for index, text in enumerate(text.split("\n")):
    print(text, " : ", predict[index])

What is not to like about this product.  :  negative
Not bad.  :  negative
Not an issue.  :  negative
Not buggy.  :  neutral
Not happy.  :  positive
Not user-friendly.  :  negative
Not good.  :  positive
Is it any good?  :  positive
I do not dislike horror movies.   :  negative
Disliking horror movies is not uncommon.   :  negative
Sometimes I really hate the show.   :  negative
I love having to wait two months for the next series to come out!   :  positive
The final episode was surprising with a terrible twist at the end.  :  neutral
The film was easy to watch but I would not recommend it to my friends.   :  neutral
I LOL’d at the end of the cake scene.  :  neutral


**Ex 3 :Application: Extractive Text Summarization**


Task:
▪ Explain the pipeline for summarizing an input document by identifying informative sentences.
▪ Perform cleaning and preprocessing of text.
▪ Generate representations using:
• Bag of Words
• TF-IDF
• Continuous Bag of Words
• Skip gram
• Word2Vec.
• GloVe
• FastText
▪ Generate representations for each sentence using the above word embedding techniques.
▪ Identify the distance between each sentence. Evaluate if semantically similar sentences lie
closer to each other.
▪ Explain the impact of embedding techniques in identifying the distance between sentences.
▪ Which Embedding technique generates contextual representations? Justify.

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
from nltk.tokenize import sent_tokenize

def pre_processing(text):

    # text to sentence
    tokenized = sent_tokenize(text)

    # Remove Punctuation
    # Lower Case
    # Strip White Spaces
    pattern   = re.compile(r'[^a-zA-Z0-9\s]')
    tokenized = [pattern.sub('', sent).strip().lower() for sent in tokenized]

    return tokenized

corpus = []
for doc in data:
    corpus.extend(pre_processing(doc))

print("Number of Sentences in Corpus : ", len(corpus))

NameError: name 'data' is not defined

**Exercise 4**





Application: Hate Speech Identification
Dataset: Download the dataset from Hate Speech and Offensive Language Dataset (kaggle.com)
Task:
▪ Explain the pipeline for developing sentiment analysis task.
▪ Perform cleaning and preprocessing of text.
▪ Generate representations using:
• Word2Vec
• Fasttext
• CNN.
• RNN
▪ Feed the representations as input to appropriate classification technique.
▪ Evaluate the accuracy of each model and analyze the labels that are misclassified.

In [ ]:
# Import Library

import re
import nltk
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Dense

from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *


In [ ]:
data = pd.read_csv(r'/content/Hate speech.csv')

In [ ]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
data = data[['tweet','class']]

In [ ]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   24783 non-null  object
 1   class   24783 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 387.4+ KB


In [ ]:
!pip install emoji


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
import emoji

lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess_tweet(tweet):
    tokens = tokenizer.tokenize(tweet)
    tokens = [token.lower() for token in tokens]
    tokens = [re.sub(r'https?://\S+|www\.\S+', '', token) for token in tokens]
    tokens = [re.sub(r'\W', ' ', token) for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [emoji.demojize(token) for token in tokens]
    contractions = {
        "can't": "cannot",
        "won't": "will not",
    }
    tokens = [contractions[token] if token in contractions else token for token in tokens]
    tokens = [token for token in tokens if not token.startswith('@')]
    processed_tweet = ' '.join(tokens)

    return processed_tweet


In [ ]:
preprocess_tweet("@Dia jsut poseted someting (*)@(*$)(*)#@)!!! Im screaming ^U^😃")

'jsut poseted someting im screaming u'

In [ ]:
data['pre_process_tweet'] = data['tweet'].apply(preprocess_tweet)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tweet              24783 non-null  object
 1   class              24783 non-null  int64 
 2   pre_process_tweet  24783 non-null  object
dtypes: int64(1), object(2)
memory usage: 581.0+ KB


In [ ]:
data.head()

,tweet,class,pre_process_tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...,2,rt woman complain cleaning house man always ta...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1,rt boy dat cold tyga dwn bad cuffin dat hoe 1s...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,rt dawg rt 0sbaby4life ever fuck bitch start c...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1,rt look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,rt shit hear might true might faker bitch told ya


In [ ]:
#Generating representations

#WordtoVec: Skipgram and CBOW

from gensim.models import Word2Vec
import numpy as np

# Assuming cbow is already trained
cbow = Word2Vec(data['pre_process_tweet'].values.tolist(), vector_size=100)

def get_mean_vector(model, sentence, vocab):
    words = [word for word in sentence if word in vocab]
    if len(words) >= 1:
        return np.mean(model.wv[words], axis=0)
    return np.zeros((100,))

vocab = cbow.wv.index_to_key

cbow_array = []
for sentence in data['pre_process_tweet'].values.tolist():
    cbow_array.append(get_mean_vector(cbow, sentence, vocab))

print(len(cbow_array))



24783


In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Assuming skipgram is already trained
skipgram = Word2Vec(data['pre_process_tweet'].values.tolist(), vector_size=100, sg=1)  # sg=1 specifies SkipGram

def get_mean_vector(model, sentence, vocab):
    words = [word for word in sentence if word in vocab]
    if len(words) >= 1:
        return np.mean(model.wv[words], axis=0)
    return np.zeros((100,))

vocab = skipgram.wv.index_to_key

skipgram_array = []
for sentence in data['pre_process_tweet'].values.tolist():
    skipgram_array.append(get_mean_vector(skipgram, sentence, vocab))

print(len(skipgram_array))


24783


In [ ]:
data

,tweet,class,pre_process_tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...,2,rt woman complain cleaning house man always ta...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1,rt boy dat cold tyga dwn bad cuffin dat hoe 1s...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,rt dawg rt 0sbaby4life ever fuck bitch start c...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1,rt look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,rt shit hear might true might faker bitch told ya
...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1,muthaf lie right tl trash mine bible scripture...
24779,"you've gone and broke the wrong heart baby, an...",2,gone broke wrong heart baby drove redneck crazy
24780,young buck wanna eat!!.. dat nigguh like I ain...,1,young buck wanna eat dat nigguh like aint fuck...
24781,youu got wild bitches tellin you lies,1,youu got wild bitch tellin lie


In [ ]:
tweets = data['pre_process_tweet'].values
labels = data['class'].values

#Tokenizing and Padding

max_len = data['pre_process_tweet'].str.len().max()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)
tweet_data = pad_sequences(sequences,maxlen=max_len)


In [ ]:
#Model Building

vocab_size = len(tokenizer.word_index) + 1
cnn_representation_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, cnn_representation_dim, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics= 'accuracy')
model.fit(tweet_data, labels, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
620/620 [==============================] - 39s 62ms/step - loss: 0.3439 - accuracy: 0.8773 - val_loss: 0.2513 - val_accuracy: 0.9161
Epoch 2/10
620/620 [==============================] - 38s 61ms/step - loss: 0.1949 - accuracy: 0.9284 - val_loss: 0.2959 - val_accuracy: 0.9030
Epoch 3/10
620/620 [==============================] - 37s 59ms/step - loss: 0.0933 - accuracy: 0.9670 - val_loss: 0.3330 - val_accuracy: 0.9042
Epoch 4/10
620/620 [==============================] - 43s 69ms/step - loss: 0.0489 - accuracy: 0.9838 - val_loss: 0.3790 - val_accuracy: 0.8973
Epoch 5/10
620/620 [==============================] - 36s 58ms/step - loss: 0.0332 - accuracy: 0.9886 - val_loss: 0.4543 - val_accuracy: 0.9074
Epoch 6/10
620/620 [==============================] - 36s 58ms/step - loss: 0.0252 - accuracy: 0.9914 - val_loss: 0.4508 - val_accuracy: 0.8880
Epoch 7/10
620/620 [==============================] - 38s 61ms/step - loss: 0.0208 - accuracy: 0.9921 - val_loss: 0.5586 - val_accuracy:

In [ ]:
cnn_embeddings = model.layers[0].get_weights()[0]
cnn_embeddings

array([[ 0.01696684, -0.00994224, -0.01948774, ..., -0.04631813,
         0.00565956,  0.03472973],
       [ 0.11805116,  0.17424363,  0.1470484 , ..., -0.04032356,
        -0.09028555, -0.06763017],
       [-0.00037258,  0.0233454 ,  0.03723885, ..., -0.02918104,
         0.07457133,  0.0147731 ],
       ...,
       [-0.00192814,  0.00901241, -0.00971892, ...,  0.00437753,
         0.00126193, -0.00257976],
       [-0.02485954, -0.02839837, -0.04000815, ...,  0.00608123,
        -0.00813431, -0.02873926],
       [ 0.03607332, -0.02326905, -0.01823951, ..., -0.00224997,
        -0.04690826, -0.0413673 ]], dtype=float32)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

def train_and_evaluate_decision_tree(x_train, x_test, y_train, y_test, representation):
    dtclassifier = DecisionTreeClassifier(random_state=9, max_depth=5)
    dtclassifier.fit(x_train, y_train)
    y_pred = dtclassifier.predict(x_test)
    print(f"\nMetrics for {representation}:")
    print(f"Model Score: {dtclassifier.score(x_train, y_train)}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

def get_dataset_prep(array, labels, test_size=0.2):  # Added default test_size
    x_train, x_test, y_train, y_test = train_test_split(array, labels, test_size=test_size)
    return x_train, x_test, y_train, y_test


In [ ]:
x_train,x_test,y_train,y_test = get_dataset_prep(cbow_array,labels)
train_and_evaluate_decision_tree(x_train, x_test, y_train, y_test, "DT-CBOW")


Metrics for DT-CBOW:
Model Score: 0.7882578432361546
Accuracy: 0.7700221908412346
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.03      0.05       312
           1       0.80      0.95      0.87      3805
           2       0.47      0.25      0.33       840

    accuracy                           0.77      4957
   macro avg       0.69      0.41      0.41      4957
weighted avg       0.74      0.77      0.72      4957



In [ ]:
x_train,x_test,y_train,y_test = get_dataset_prep(skipgram_array,labels)
train_and_evaluate_decision_tree(x_train, x_test, y_train, y_test, "DT-Skip")



Metrics for DT-Skip:
Model Score: 0.7731261979219207
Accuracy: 0.7837401654226347
Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.03      0.05       272
           1       0.79      1.00      0.88      3889
           2       0.20      0.00      0.00       796

    accuracy                           0.78      4957
   macro avg       0.46      0.34      0.31      4957
weighted avg       0.67      0.78      0.69      4957



In [ ]:
#RNN representations
from tensorflow.keras.layers import SimpleRNN  # Importing SimpleRNN from Keras

vocab_size = len(tokenizer.word_index) + 1
rnn_representation_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, rnn_representation_dim, input_length=max_len))
model.add(SimpleRNN(32,return_sequences=False))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = "accuracy")


In [ ]:
model.fit(tweet_data, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
620/620 [==============================] - 36s 55ms/step - loss: 0.4093 - accuracy: 0.8556 - val_loss: 0.2752 - val_accuracy: 0.9098
Epoch 2/10
620/620 [==============================] - 36s 58ms/step - loss: 0.1966 - accuracy: 0.9303 - val_loss: 0.2895 - val_accuracy: 0.9038
Epoch 3/10
620/620 [==============================] - 31s 50ms/step - loss: 0.0837 - accuracy: 0.9721 - val_loss: 0.3845 - val_accuracy: 0.8925
Epoch 4/10
620/620 [==============================] - 33s 53ms/step - loss: 0.0453 - accuracy: 0.9848 - val_loss: 0.4555 - val_accuracy: 0.8804
Epoch 5/10
620/620 [==============================] - 33s 53ms/step - loss: 0.0320 - accuracy: 0.9892 - val_loss: 0.5304 - val_accuracy: 0.8531
Epoch 6/10
620/620 [==============================] - 32s 52ms/step - loss: 0.0279 - accuracy: 0.9901 - val_loss: 0.5564 - val_accuracy: 0.8713
Epoch 7/10
620/620 [==============================] - 35s 57ms/step - loss: 0.0239 - accuracy: 0.9915 - val_loss: 0.5636 - val_accuracy:

In [ ]:
rnn_embeddings = model.layers[0].get_weights()[0]
rnn_embeddings

array([[-0.01581835,  0.02156218, -0.06654426, ..., -0.01921615,
         0.00106114, -0.0058489 ],
       [-0.01933762, -0.00727147,  0.169275  , ...,  0.11585952,
        -0.22337076,  0.14639181],
       [ 0.03624801,  0.02861097, -0.00672321, ..., -0.00611927,
         0.02496484, -0.04060837],
       ...,
       [-0.03060452,  0.00888643,  0.03988672, ..., -0.04953717,
         0.04128316,  0.02194059],
       [-0.01201225, -0.04464687,  0.02023626, ...,  0.01762203,
        -0.02263118,  0.01549162],
       [ 0.01737351, -0.00876598, -0.00832968, ...,  0.01395141,
        -0.04519268,  0.04634757]], dtype=float32)

**Exercise 5**

Application: Hate Speech Identification
Dataset: Download the dataset from Annotated Corpus for Named Entity Recognition (kaggle.com)
Task:
• Develop a model for Named Entity Recognition using Hidden Markov Model.
• Remove the labels from the Corpus and use Baum Welch algorithm to estimate the learning
parameters.

In [ ]:
import pandas as pd

### **Exercise 6**

Application: Keyword Extraction
Dataset: Download the dataset from SemEval2017
Task:
• Develop a model for identifying and extracting keywords/ key phrases from the input sequence.
• Use Named Entity Recognition and PoS tagging as a feature that contribute to the extraction.

### **Exercise 8**

Application: Machine Translation

Dataset: Download the dataset from
Tamil-English-Dataset/Dataset at master · Ishikahooda/Tamil-English-Dataset · GitHub
English_to_tamil_data (kaggle.com)

Task:
• Develop a model that can translate a Tamil input sentence to English.
• Analyze the impact of various embedding techniques in the translation task.
• Use sequence model to translate the Tamil input sentence and analyze the performance of
various techniques.
• Translate the input sentences using Transformer model and understand its functioning.
• Translate the following Tamil Phrase and evaluate your model based on the expected output.


Input 1: நான் மிகவும் சந்த ாஷமாக இருக்கிதேன்
Expected Output: Im so happy
Input 2: அது அவசியமில்லை
Expected Output: It wasnt necessary
Input 3: யவுசசய்து அல மீண்டும் சசய்யவும்
Expected Output: Please do that again
Input 4: அது ஒரு நல்ை தயாசலை
Expected Output: That is a good idea
Input 5: அவர்கள் ஒன்ோக தவலை சசய்ய ஒப்புக்சகாண்டைர்
Expected Output: They agreed to work together

https://alvinntnu.github.io/python-notes/nlp/seq-to-seq-machine-translation.html

https://www.tensorflow.org/text/tutorials/transformer

In [ ]:
with open("/content/tam.txt", encoding="utf8") as f:
    data = f.read().split("\n")

X_txt = []
y_txt = []

X_voc = set()
y_voc = set()

for line in data:
    # Skip In-Valid Seq
    if(len(line.split("\t")) != 3):
        continue

    target, inp, _ = line.split("\t")
    target = "\t" + target + "\n" # "\t" -> Start Seq, "\n" -> End Seq

    X_txt.append(inp)
    y_txt.append(target)

    X_voc.update(set(inp))
    y_voc.update(set(target))

In [ ]:
print("No of Records : ", len(X_txt), end="\n\n")

X_voc = sorted(list(X_voc))
y_voc = sorted(list(y_voc))

print("Vocab Size of Input tok : ", len(X_voc))
print("vocab Size of Output tok : ", len(y_voc), end="\n\n")

max_encoder = max([len(t) for t in X_txt])
max_decoder = max([len(t) for t in y_txt])

print("Max Seq length for input : ", max_encoder)
print("Max Seq length for output : ", max_decoder)

No of Records :  201

Vocab Size of Input tok :  52
vocab Size of Output tok :  55

Max Seq length for input :  109
Max Seq length for output :  96


In [ ]:
# Tokenisation

input_tok_enc = dict([(char, i) for i, char in enumerate(X_voc)])
input_tok_dec = dict([(i, char) for i, char in enumerate(X_voc)])

target_tok_enc = dict([(char, i) for i, char in enumerate(y_voc)])
target_tok_dec = dict([(i, char) for i, char in enumerate(y_voc)])
X = [[input_tok_enc[char] for char in text] for text in X_txt]
y = [[target_tok_enc[char] for char in text] for text in y_txt]

In [ ]:
#Padding

from tensorflow.keras.utils import pad_sequences

X = pad_sequences(X, maxlen=max_encoder, padding="post", value=input_tok_enc[' '])
y = pad_sequences(y, maxlen=max_decoder, padding="post", value=target_tok_enc[' '])

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, CategoryEncoding

latent_dim = 256

# Encoder Block
encoder_inputs = Input(shape=(None, len(X_voc)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

# Decoder Block
decoder_inputs = (Input(shape=(None, len(y_voc))))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(y_voc), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, None, 52)]           0         []                            
                                                                                                  
 input_14 (InputLayer)       [(None, None, 55)]           0         []                            
                                                                                                  
 lstm_8 (LSTM)               [(None, 256),                316416    ['input_13[0][0]']            
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                           

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder_input_data = np.zeros((len(X), max_encoder, len(X_voc)), dtype="float")
for i, line in enumerate(X):
    for j, k in enumerate(line):
        encoder_input_data[i][j][k] = 1.0

decoder_input_data = np.zeros((len(X), max_decoder, len(y_voc)), dtype="float")
decoder_target_data = np.zeros((len(X), max_decoder, len(y_voc)), dtype="float")
for i, line in enumerate(y):
    for j, k in enumerate(line):
        decoder_input_data[i, j, k] = 1.0
        if(j > 0):
            decoder_target_data[i, j-1, k] = 1.0

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=500, validation_split=0.2)

Epoch 1/500
3/3 [==============================] - 4s 1s/step - loss: 0.4889 - accuracy: 0.8553 - val_loss: 1.4174 - val_accuracy: 0.6664
Epoch 2/500
3/3 [==============================] - 3s 1s/step - loss: 0.4858 - accuracy: 0.8579 - val_loss: 1.3808 - val_accuracy: 0.6723
Epoch 3/500
3/3 [==============================] - 3s 867ms/step - loss: 0.4841 - accuracy: 0.8571 - val_loss: 1.3818 - val_accuracy: 0.6669
Epoch 4/500
3/3 [==============================] - 3s 924ms/step - loss: 0.4864 - accuracy: 0.8561 - val_loss: 1.3403 - val_accuracy: 0.6662
Epoch 5/500
3/3 [==============================] - 2s 593ms/step - loss: 0.4846 - accuracy: 0.8574 - val_loss: 1.3713 - val_accuracy: 0.6695
Epoch 6/500
3/3 [==============================] - 2s 599ms/step - loss: 0.4877 - accuracy: 0.8585 - val_loss: 1.4119 - val_accuracy: 0.6659
Epoch 7/500
3/3 [==============================] - 2s 598ms/step - loss: 0.4948 - accuracy: 0.8568 - val_loss: 1.3253 - val_accuracy: 0.6715
Epoch 8/500
3/3 [==

In [ ]:
#model.save('tam2eng.keras')


In [ ]:
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # LSTM 1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]  # LSTM 2
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]  # Dense Layer
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

def decode_seq(input_seq):
    # Encoder
    state_value = encoder_model.predict(input_seq, verbose=0)

    # Decoder
    target_seq = np.zeros((1, 1, len(y_voc)))
    target_seq[0, 0, target_tok_enc["\t"]] = 1.0
    stop = False
    decode_sent = ""
    while not stop:
        output_tok, h, c = decoder_model.predict([target_seq] + state_value, verbose=0)
        sample_token_idx = np.argmax(output_tok[0, -1, :])
        sample_char = target_tok_dec[sample_token_idx]
        decode_sent += sample_char
        if sample_char == "\n" or len(decode_sent) > max_decoder:
            stop = True
        target_seq = np.zeros((1, 1, len(y_voc)))
        target_seq[0, 0, sample_token_idx] = 1.0
        state_value = [h, c]
    return decode_sent

for i in range(10):
    input_seq = encoder_input_data[i: i+1]
    d_seq = decode_seq(input_seq)
    print("Input Seq : ", X_txt[i])
    print("Output Seq : ", d_seq)


Input Seq :  நான் தூங்கினேன்.
Output Seq :  I want to go han.

Input Seq :  அமைதியாக இருங்கள்
Output Seq :  I want to go abarread.

Input Seq :  நான் நடப்பேன்.
Output Seq :  I want to go abrare.

Input Seq :  அவன் யார்?
Output Seq :  I want to go abarread.

Input Seq :  யாருக்குத் தெரியும்?
Output Seq :  I want to go abartof.

Input Seq :  அவள் சிரித்தாள்
Output Seq :  I want to go abarread.

Input Seq :  என்னிடம் பேசு
Output Seq :  I want to go gon.

Input Seq :  அவள் யார்?
Output Seq :  I want to go abarread.

Input Seq :  போய் தூங்கு
Output Seq :  I want to go ang.

Input Seq :  மழை பெய்யலாம்
Output Seq :  I want to go abartof.

